# Language Transcription and Detection Using Whisper

This notebook uses the OpenAI Whisper model (https://github.com/openai/whisper) to convert audio files into text. The output provides language categorization together with a confidence score. 

We run two versions of transcription:
* Transcribe text
* Transcibe text with pause annotation 


## Setup

In [2]:
import os
#import torch
import csv
import whisper
#import pandas as pd
from pydub import AudioSegment, effects
from tqdm import tqdm
import pandas as pd

## Functions

In [34]:
def process_audio(file_path):
    """Process the audio to have a consistent and amplified volume level."""
    audio = AudioSegment.from_file(file_path)
    audio = effects.normalize(audio)  # Normalize audio
    audio = audio + 10
    return audio


def transcribe_audio_with_language(file_path, model):
    """
    Transcribe the audio and detect language using the whisper model after normalizing.
    Returns transcription, detected language, and confidence.
    """

    # Process audio
    processed_audio = process_audio(file_path)
    processed_path = file_path.replace(".mp3", "_processed.wav")
    processed_audio.export(processed_path, format="wav")

    # Load and preprocess audio
    audio = whisper.load_audio(processed_path)
    audio = whisper.pad_or_trim(audio)
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # Language detection
    _, probs = model.detect_language(mel)
    detected_lang = max(probs, key=probs.get)
    confidence = probs[detected_lang]
    
    # Transcription
    transcription = model.transcribe(processed_path)["text"]
    
    return transcription, detected_lang, confidence



def transcribe_audio_with_pause(file_path, model, pause_threshold=0.05):
    """
    Transcribe the audio and detect language using the whisper model after normalizing.
    Adds sentence-level pause markers to the transcription.
    
    Args:
        file_path (str): Path to the audio file.
        model: Whisper model instance.
        pause_threshold (float): Minimum duration of silence (in seconds) to add a pause marker.
    
    Returns:
        tuple: Transcription with pauses, detected language, and confidence.
    """
    # Process audio
    #processed_audio = process_audio(file_path)
    #processed_path = file_path.replace(".mp3", "_processed.wav")
    #processed_audio.export(processed_path, format="wav")

    #processed_path = file_path.replace(".mp3", "_processed.wav") #need to change
    processed_path = file_path

    # Load and preprocess audio
    audio = whisper.load_audio(processed_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # Language detection
    _, probs = model.detect_language(mel)
    detected_lang = max(probs, key=probs.get)
    confidence = probs[detected_lang]
    
    # Transcription with timestamps
    result = model.transcribe(processed_path, word_timestamps=True)
    segments = result.get("segments", [])
    
    # Add pauses to the transcription
    # transcript_with_pauses = []
    # for i, segment in enumerate(segments):
    #     transcript_with_pauses.append(segment["text"].strip())
    #     if i < len(segments) - 1:
    #         # Calculate pause duration
    #         current_end = segment["end"]
    #         next_start = segments[i + 1]["start"]
    #         pause_duration = next_start - current_end
            
    #         # Add pause marker if pause exceeds threshold
    #         if pause_duration >= pause_threshold:
    #             transcript_with_pauses.append(f"[PAUSE: {pause_duration:.1f}s]")
    annotated_transcription = []
    for i, segment in enumerate(segments):
        # Annotate word-level pauses within the segment
        words = segment.get("words", [])
        for j, word in enumerate(words[:-1]):  # Exclude the last word initially
            annotated_transcription.append(word["word"])  # Append the word
            current_end = word["end"]
            next_start = words[j + 1]["start"]
            word_pause = next_start - current_end
            if word_pause >= pause_threshold:
                annotated_transcription.append(f"[WORD PAUSE: {word_pause:.1f}s]")

        # Append the last word of the segment if it exists
        if words:
            annotated_transcription.append(words[-1]["word"])

        # Annotate sentence-level pause between segments
        if i < len(segments) - 1:
            current_end = segment["end"]
            next_start = segments[i + 1]["start"]
            sentence_pause = next_start - current_end
            if sentence_pause >= pause_threshold:
                annotated_transcription.append(f"[SENTENCE PAUSE: {sentence_pause:.1f}s]")

    # Combine words into a single string without adding extra spaces
    transcription = "".join(annotated_transcription)
    return transcription, detected_lang, confidence


## Load model

In [4]:
whisper_model = whisper.load_model("small")

/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=devi

## Test cases

In [9]:
transcribe_audio_with_language("data/raw_data/train_audios_sample/cyqq.mp3", whisper_model)

/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(" of the house with bushes and another window, a sidewalk, tree, a roof of the house, the side of the house, the curtains on the kitchen window, I assume that's mother, wiping dishes with the color top and the sink and the faucets.",
 'en',
 0.9743459224700928)

In [38]:
transcribe_audio_with_language("data/raw_data/train_audios_sample/jgcl.mp3", whisper_model)

/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(' Em un lugar de la mancha, de cuyo nombre non quiera acordarme, no mocho tiempo que vivía unida algo de los de lanza en astillero, a dar gantigua, rocin flaco e galgo corredor. Una olla de algo máis vaca que carnero. Salpicón las máis noches, duelos e querrantos rosados, lentejas los viernes, algún palomíno de añadidura los domingos, consumían las tres partes de sua cienda.',
 'gl',
 0.7763605117797852)

In [6]:
transcribe_audio_with_language("data/raw_data/train_audios_sample/keec.mp3", whisper_model)

/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(" Okay, the child reaching for a cookie. The chair is tilted, the girl is possibly moving there. And the egg was removed and reaching for a cookie. Water is running out of the sink, another is drying dishes, and there's going to be screaming. The kid falls.",
 'en',
 0.9885423183441162)

In [37]:
transcribe_audio_with_language("data/raw_data/train_audios_sample/delk.mp3", whisper_model)

/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(" There it is. There's a boy trying to get some cookies and a cupboard. There's a stool that's ready to fall that he's standing on. There's a lady washing dishes and she's stowed. And there's a sink that's stowed up full of water and it's stowed all on the floor. She's standing by the window with her teargrass and she's shrapneling outside. Two cups and a plate to be laying on the sink.",
 'en',
 0.99535071849823)

In [32]:
transcribe_audio_with_pause("data/raw_data/train_audios_sample/delk.mp3", whisper_model)

/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


(" There it is.[SENTENCE PAUSE: 0.1s] There's a boy trying to get some cookies and a cupboard.[SENTENCE PAUSE: 0.4s] There's a stool that's ready to fall that he's standing on.[SENTENCE PAUSE: 0.1s] There's a lady washing dishes and she's stowed.[SENTENCE PAUSE: 0.3s] And there's a sink that's stowed up full of water and it's stowed all on the floor.[SENTENCE PAUSE: 0.3s] She's standing by the window with her teargrass and she's shrapneling outside.[SENTENCE PAUSE: 2.1s] Two cups and a plate to be laying on the sink.",
 'en',
 0.99535071849823)

## Transcribe

In [10]:
## loop through target file directory

# Specify the directory path
directory_path = "data/raw_data/train_audios_sample"

# Specify the output CSV file
output_path = "data/produced_data/transcript_train_audios_sample_small.csv"

# Open the CSV file for writing
with open(output_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write the header
    csv_writer.writerow(['filename', 'transcription', 'langage', 'confidence'])

    # Get a list of raw files
    raw_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3')]

    for filename in tqdm(raw_files, desc="Processing audio files"):
    #for filename in raw_files:
        file_path = os.path.join(directory_path, filename)
        print(filename, file_path)

        # transcript file
        transcription, language, confidence = transcribe_audio_with_language(file_path, whisper_model)
        
        # Write the filename to the CSV
        csv_writer.writerow([os.path.splitext(filename)[0], transcription, language, confidence])

Processing audio files:   0%|          | 0/15 [00:00<?, ?it/s]

sivi.mp3 data/raw_data/train_audios_sample/sivi.mp3


/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:   7%|▋         | 1/15 [00:06<01:34,  6.73s/it]

uxwg.mp3 data/raw_data/train_audios_sample/uxwg.mp3


/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:  13%|█▎        | 2/15 [00:12<01:22,  6.34s/it]

...

/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:  87%|████████▋ | 13/15 [01:27<00:12,  6.33s/it]

ojvu.mp3 data/raw_data/train_audios_sample/ojvu.mp3


/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:  93%|█████████▎| 14/15 [01:34<00:06,  6.55s/it]

slet.mp3 data/raw_data/train_audios_sample/slet.mp3


/opt/anaconda3/envs/prepare/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|██████████| 15/15 [01:44<00:00,  6.96s/it]


In [16]:
## loop through target file directory

# Specify the directory path
directory_path = "data/raw_data/test_audios"

# Specify the output CSV file
output_path = "data/produced_data/transcript_test_audios_small_v0.csv"

# Check transcribed samples
# df_transcribe = pd.read_csv("data/produced_data/transcript_train_audios_small_v0.csv")
# sample_list = list(df_transcribe['filename'].unique())

# Open the CSV file for writing
with open(output_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write the header
    csv_writer.writerow(['filename', 'transcription', 'langage', 'confidence'])

    # Get a list of raw files
    raw_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3')]

    for filename in tqdm(raw_files, desc="Processing audio files"):
    #for filename in raw_files:
        # if os.path.splitext(filename)[0] in sample_list:
        #     "already transcribed!"
        #     continue
        file_path = os.path.join(directory_path, filename)
        print(filename, file_path)

        # transcript file
        transcription, language, confidence = transcribe_audio_with_language(file_path, whisper_model)
        
        # Write the filename to the CSV
        csv_writer.writerow([os.path.splitext(filename)[0], transcription, language, confidence])

Processing audio files:   0%|          | 0/412 [00:00<?, ?it/s]

yrhm.mp3 data/raw_data/test_audios/yrhm.mp3


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:   0%|          | 1/412 [00:05<37:35,  5.49s/it]

zbdh.mp3 data/raw_data/test_audios/zbdh.mp3


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:   0%|          | 2/412 [00:11<40:30,  5.93s/it]

...

/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|█████████▉| 410/412 [46:52<00:13,  6.90s/it]

pedu.mp3 data/raw_data/test_audios/pedu.mp3


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|█████████▉| 411/412 [47:00<00:07,  7.30s/it]

zrzh.mp3 data/raw_data/test_audios/zrzh.mp3


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|██████████| 412/412 [47:06<00:00,  6.86s/it]


## Transcribe with pause annotation

In [38]:
## loop through target file directory

# Specify the directory path
directory_path = "data/raw_preprocessed/test_audios"

# Specify the output CSV file
output_path = "data/transcripts/transcript_test_audios_small_v2.csv"

# Check transcribed samples
# df_transcribe = pd.read_csv("data/produced_data/transcript_train_audios_small_v0.csv")
# sample_list = list(df_transcribe['filename'].unique())

# Open the CSV file for writing
with open(output_path, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write the header
    csv_writer.writerow(['filename', 'transcription', 'language', 'confidence'])

    # Get a list of raw files
    #raw_files = [f for f in os.listdir(directory_path) if f.endswith('.mp3')] #need to change

    raw_files = [f for f in os.listdir(directory_path) if f.endswith('.wav')]

    for filename in tqdm(raw_files, desc="Processing audio files"):
    #for filename in raw_files:
        # if os.path.splitext(filename)[0] in sample_list:
        #     "already transcribed!"
        #     continue
        file_path = os.path.join(directory_path, filename)
        print(filename, file_path)

        # transcript file
        transcription, language, confidence = transcribe_audio_with_pause(file_path, whisper_model)
        
        # Write the filename to the CSV
        csv_writer.writerow([os.path.splitext(filename)[0], transcription, language, confidence])

Processing audio files:   0%|          | 0/412 [00:00<?, ?it/s]

lgid.wav data/raw_preprocessed/test_audios/lgid.wav


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:   0%|          | 1/412 [00:11<1:19:19, 11.58s/it]

ygom.wav data/raw_preprocessed/test_audios/ygom.wav


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files:   0%|          | 2/412 [00:17<58:02,  8.49s/it]  

...

/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|█████████▉| 410/412 [58:26<00:15,  7.76s/it]

swud.wav data/raw_preprocessed/test_audios/swud.wav


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|█████████▉| 411/412 [58:34<00:07,  7.87s/it]

srjb.wav data/raw_preprocessed/test_audios/srjb.wav


/opt/anaconda3/envs/prepare/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing audio files: 100%|██████████| 412/412 [58:41<00:00,  8.55s/it]
